In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from pathlib import Path
from time import strftime

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [3]:
def get_run_logdir(root_logdir='my_logs'):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100,200))

In [3]:
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mse', optimizer=optimizer, metrics=['RootMeanSquaredError'])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 0.9308 - root_mean_squared_error: 0.9648 - val_loss: 0.6263 - val_root_mean_squared_error: 0.7914
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3898 - root_mean_squared_error: 0.6244 - val_loss: 1.9747 - val_root_mean_squared_error: 1.4052
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3703 - root_mean_squared_error: 0.6085 - val_loss: 0.3844 - val_root_mean_squared_error: 0.6200
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3457 - root_mean_squared_error: 0.5880 - val_loss: 0.8634 - val_root_mean_squared_error: 0.9292
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3481 - root_mean_squared_error: 0.5900 - val_loss: 0.3408 - val_root_mean_squared_error: 0.5838
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3199 - root_mean_squared_error: 0.5656 - val_loss: 0.3587 - val_

In [4]:
import numpy as np

test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(str(test_logdir))
with writer.as_default():
    for step in range(1,1000+1):
        tf.summary.scalar('my_scalar', np.sin(step/10), step=step)

        data = (np.random.randn(100) + 2 * step / 100)
        tf.summary.histogram('my_hist', data, buckets=50, step=step)

        images = np.random.randn(2, 32, 32, 3) * step / 1000
        tf.summary.image('my_images', images, step=step)

        texts = ['Krok wynosi ' + str(step), 'Jego kwadrat to ' + str(step ** 2)]
        tf.summary.text('my_text', texts, step=step)

        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio('my_audio', audio, sample_rate=48000, step=step)